In [ ]:
#!pip install gurobipy pandas openpyxl
#!pip install gdown


   ----------------------------------------  0/12 [urllib3]
   --- ------------------------------------  1/12 [typing-extensions]
   ------ ---------------------------------  2/12 [tqdm]
   ------ ---------------------------------  2/12 [tqdm]
   ---------- -----------------------------  3/12 [soupsieve]
   ---------------- -----------------------  5/12 [idna]
   ----------------------- ----------------  7/12 [charset_normalizer]
   ------------------------------ ---------  9/12 [requests]
   --------------------------------- ------ 10/12 [beautifulsoup4]
   --------------------------------- ------ 10/12 [beautifulsoup4]
   --------------------------------- ------ 10/12 [beautifulsoup4]
   --------------------------------- ------ 10/12 [beautifulsoup4]
   ---------------------------------------- 12/12 [gdown]



In [1]:
import pandas as pd
import gdown
from gurobipy import Model, GRB, quicksum

In [93]:
#Descargamos el archivo desde Drive
file_id = '1BNagy0v4Nwg8dCyvZdS9rZOaR2-L9ROL'
output = 'INPUT MODELO OPTIMIZACIÓN.xlsx'
gdown.download(f"https://drive.google.com/uc?id={file_id}", output, quiet=False)


Downloading...
From: https://drive.google.com/uc?id=1BNagy0v4Nwg8dCyvZdS9rZOaR2-L9ROL
To: e:\proyectos\optimizacion_TDG\tdg2_mcd\INPUT MODELO OPTIMIZACIÓN.xlsx
100%|██████████| 364k/364k [00:00<00:00, 1.42MB/s]


'INPUT MODELO OPTIMIZACIÓN.xlsx'

In [129]:
# ==== PREPARACIÓN DE DICCIONARIOS PARA CONSTRUIR CONJUNTOS Y PARÁMETROS ===
dias = 12 #Días de corrida del modelo
PENALIZACION = 10000 #Penalización del incumplimiento del mínimo de la desviación

# === 1. LEER DATOS DE ENTRADA ===
archivo = "INPUT MODELO OPTIMIZACIÓN.xlsx"
df_granjas = pd.read_excel(archivo, sheet_name="Input_Granjas")
df_demanda = pd.read_excel(archivo, sheet_name="Input_Demanda", header=4)
df_raw = pd.read_excel(archivo, sheet_name="Input_Demanda", header=None)

# === PARÁMETROS ===
W, ADG, CAT, N, TIPO = {}, {}, {}, {}, {}

for _, row in df_granjas.iterrows():
    g, p, s = row["GRANJA"], row["GALPON"], row["SEXO"]
    N[(g, p, s)] = row["SALDO"]
    TIPO[(g, p, s)] = row["TIPO"]

    for d in range(1, 1 + dias):
        col_w = 'WEIGTH' if d == 1 else f'WEIGTH{d}'
        col_adg = 'ADG' if d == 1 else f'ADG{d}'
        col_cat = 'CAT PESO' if d == 1 else f'CAT PESO{d}'

        if pd.notnull(row[col_w]): W[(g, p, s, d)] = float(row[col_w])
        if pd.notnull(row[col_adg]): ADG[(g, p, s, d)] = float(row[col_adg])
        if pd.notnull(row[col_cat]): CAT[(g, p, s, d)] = row[col_cat].strip().upper()

# === DEMANDA Q[tipo, cat, d] ===
Q = {}
tipos_validos = ["CAMPESINO", "BLANCO"]
cats_validas = ["PEQUEÑO", "MEDIANO", "GRANDE"]

for _, row in df_demanda.iterrows():
    t, c = row["Tipo"], row["Categoria Peso"]
    if t in tipos_validos and c in cats_validas:
        for d in range(1, 1 + dias):
            val = row.get(f"Day{d}")
            if pd.notnull(val):
                Q[(t, c, d)] = val

# === MAPD y TOT ===
MAPD = {}

for i in range(13, 13 + dias):
    semana = df_raw.iloc[i, 1]
    dia = df_raw.iloc[i, 2]
    if pd.notnull(semana) and pd.notnull(dia):
        MAPD[int(dia)] = int(semana)

TOT = {}
for d in range (1, 1 + dias):
    s = MAPD.get(d) #Se trae la semana a la que pertenece cada uno de los días
    val_tot = df_raw.iloc[14 + s, 5] #La demanda de la semana está alacenado de la fila 16 en adelante en la columna 5. Se asegura de que se trae la demanda según la semana
    if pd.notnull(val_tot): TOT[s] = float(val_tot)

tol = float(df_raw.iloc[1, 2])  # Tolerancia


In [130]:
# === 2. DEFINICIÓN DE CONJUNTOS ===


# Conjunto de grupos: (granja, galpón, sexo)
GP = list(N.keys())

# Conjunto de días en el horizonte (por ejemplo 1 a 6)
D = sorted(set(d for (_, _, _, d) in W.keys()))

# Conjunto de tipos de pollo
T = sorted(set(TIPO.values()))

# Conjunto de categorías de peso
C = sorted(set(c for (_, _, _, _), c in CAT.items()))

# Conjunto de semanas (extraído del mapeo día → semana)
WEEK = sorted(set(MAPD[d] for d in MAPD))


In [131]:
# === 3. CREACION DEL MODELO ===

# === MODELO VACÍO ===
model = Model("modelo_cosecha_pollos")

# === VARIABLES ===

# x[g, p, s, d] = 1 si el grupo se cosecha el día d
x = model.addVars(GP, D, vtype=GRB.BINARY, name="x")

# dev[t, c, d] = desviación absoluta entre cosecha y demanda para ese tipo, categoría y día
dev = model.addVars(T, C, D, vtype=GRB.CONTINUOUS, lb=0, name="dev")

# === Variable para la restricción suave 
violacion = model.addVars(T, C, D, name="violacion", vtype=GRB.CONTINUOUS, lb=0.0) #Almacena cuantos pollos por fuera de la desviación 


In [132]:
# === 4. PLANTEAMIENTO DE LAS RESTRICCIONES ===

# === R1: Cada grupo se cosecha 1 sola vez

for g, p, s in GP:
    model.addConstr(
        quicksum(x[g, p, s, d] for d in D) <= 1,
        name=f"R1_una_vez_{g}_{p}_{s}"
    )


# === R2: la diferencia absoluta entre lo cosechado y la demanda debe ser menor a la desviacion ===

#for t in T:
    #for c in C:
        #for d in D:
            #q_val = Q.get((t, c, d), 0)
            #if q_val > 0:
                #model.addConstr(
                    #dev[t, c, d] <= 0.15 * q_val,
                    #name=f"R2_limite_10pct_{t}_{c}_{d}"
                #)
#================================================================================================

for t in T:
    for c in C:
        for d in D:
            cantidad_real = quicksum(
                N[gps] * x[gps + (d,)] # Asegura que el grupo haya sido cosechado, y agarra la cantidad del grupo (N)
                for gps in GP
                if TIPO[gps] == t and CAT.get(gps + (d,), None) == c #Asegura que las categorias y tipos coincidan
            )
            if (t, c, d) in Q:
                model.addConstr(cantidad_real - Q[t, c, d] <= dev[t, c, d], # Modelado de la restricción en absoluto
                                name=f"R2_sup_{t}_{c}_{d}")
                model.addConstr(Q[t, c, d] - cantidad_real <= dev[t, c, d], # Modelado de la restricción en absoluto
                                name=f"R2_inf_{t}_{c}_{d}")


# === R2.2. Restricción suave que permite que se viole el mix diario de pollos

for t in T:
    for c in C:
        for d in D:
            q_val = Q.get((t, c, d), 0)
            if q_val > 0:
                model.addConstr(
                    dev[t, c, d] - tol * q_val <= violacion[t, c, d],
                    name=f"soft_limite_violacion_{t}_{c}_{d}"
                )
            else:
                # Si no hay demanda, no permitas desviación
                model.addConstr(dev[t, c, d] == 0)


In [133]:

# === R3: Para los grupos mixtos, se prioriza cosechar las hembras sobre los machos ===
galpones = set((g, p) for g, p, s in GP)

for g, p in galpones:
    if ((g, p, 'HEMBRA') in GP) and ((g, p, 'MACHO') in GP):
        for d in D:
            if d >= 5: #Solo tiene sentido si han transcurrido más de 4 días 
                suma_hembras_previos = quicksum(
                    x[g, p, 'HEMBRA', dp]
                    for dp in range(d - 4, d) if dp in D #Valida si las hembras de un galpon mixto se cosechan durante 4 días anteriores
                )
                model.addConstr(
                    x[g, p, 'MACHO', d] <= 1 - suma_hembras_previos,
                    name=f"R3_prioridad_{g}_{p}_{d}" #Evita que se cosechen los machos si no han pasado 4 días o más desde la cosecha de hembras
                )

# === R4: Asegura que la cosecha de la semana esté dentro del rango de tolerancia de la demanda total para esa semana ===

for w in WEEK:
    dias_semana = [d for d in D if MAPD.get(d) == w]
    
    suma_cosechada = quicksum(
        N[gps] * x[gps + (d,)] #Computa cuantos pollos sales por grupo cosechado
        for gps in GP
        for d in dias_semana
    )
    
    if w in TOT:
        model.addConstr(
            suma_cosechada >= (1 - tol) * TOT[w], #Límite inferior de la demanda total semanal
            name=f"R5_inferior_{w}"
        )
        model.addConstr(
            suma_cosechada <= (1 + tol) * TOT[w], #Límite superior de la demanda total semanal
            name=f"R5_superior_{w}"
        )


# === R5: No cosechar pesos por encima de 3.5 Kg ===
for (g, p, s, d), peso in W.items(): #itera sobre cada arreglo de granja, galpon, sexo y día para asegurar que el peso no se pase de 3.5 Kg
    if peso > 3.5:
        model.addConstr(
            x[g, p, s, d] == 0,
            name=f"R_peso_maximo_{g}_{p}_{s}_{d}"
        )


# === R6: El mínimo peso del pollo campesino debe ser 3 Kg ===

for (g, p, s, d), peso in W.items():
    if TIPO.get((g, p, s)) == "CAMPESINO" and peso < 3.0:
        model.addConstr(
            x[g, p, s, d] == 0,
            name=f"R_peso_min_campesino_{g}_{p}_{s}_{d}"
        )

# === R7: Solo cosechar los grupos con demanda ===

for (g, p, s, d), cat in CAT.items():
    tipo = TIPO.get((g, p, s))
    if (tipo, cat, d) not in Q:
        model.addConstr(
            x[g, p, s, d] == 0,
            name=f"R_sin_demanda_{g}_{p}_{s}_{d}"
        )


In [162]:
# === R2: Que para los grupos pequeños y medianos se cumpla la cuota diaria 

for t in T:
    for c in ["PEQUEÑO", "MEDIANO"]:  # Solo aplica a estas categorías
        for d in D:
            q_val = Q.get((t, c, d), 0)
            
            if q_val > 0:
                suma_cosechada = quicksum(
                    N[gps] * x[gps + (d,)]
                    for gps in GP
                    if TIPO[gps] == t and CAT.get(gps + (d,), None) == c
                )

                limite_inferior = (1 - tol) * q_val
                limite_superior = (1 + tol) * q_val

                # Restricción inferior
                model.addConstr(
                    suma_cosechada >= limite_inferior,
                    name=f"R2_inferior_{t}_{c}_{d}"
                )

                # Restricción superior
                model.addConstr(
                    suma_cosechada <= limite_superior,
                    name=f"R2_superior_{t}_{c}_{d}"
                )

In [163]:
# === 5. FUNCIÓN OBJETIVO ===

# MINIMIZAR la desviación diaria de cosecha con respecto a la demanda por conjuntos de pollo

model.setObjective(
    quicksum(dev[t, c, d] for t in T for c in C for d in D) +
    PENALIZACION * quicksum(violacion[t, c, d] for t in T for c in C for d in D),
    GRB.MINIMIZE
)


In [164]:
# Resolver el modelo
model.optimize()

#Revisar el estado del modelo de optimizacion
if model.status == GRB.OPTIMAL:
    print("\n✅ Solución óptima encontrada.")
elif model.status == GRB.INFEASIBLE:
    print("\n❌ El modelo es infactible.")
    model.computeIIS()
    model.write("modelo_infactible.ilp")
elif model.status == GRB.UNBOUNDED:
    print("\n❌ El modelo no está acotado.")
else:
    print(f"\n⚠️ El modelo terminó con estado: {model.status}")

Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (win64 - Windows 11.0 (22621.2))

CPU model: Intel(R) Core(TM) i7-1065G7 CPU @ 1.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Academic license 2680166 - for non-commercial use only - registered to 10___@u.icesi.edu.co
Optimize a model with 1737 rows, 3148 columns and 19456 nonzeros
Model fingerprint: 0xd7af85f2
Variable types: 208 continuous, 2940 integer (2940 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+04]
  Objective range  [1e+00, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+06]
Presolve removed 1400 rows and 1543 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -

❌ El modelo es infactible.
Academic license 2680166 - for non-commercial use

In [165]:
if model.status == GRB.INFEASIBLE:    
    model.computeIIS()

    for c in model.getConstrs():
        if c.IISConstr:
            print(f"Restricción en IIS: {c.ConstrName}")


    for v in model.getVars():
        if v.IISLB:
            print(f"Límite inferior en IIS: {v.VarName} ≥ {v.LB}")
        if v.IISUB:
            print(f"Límite superior en IIS: {v.VarName} ≤ {v.UB}")


Academic license 2680166 - for non-commercial use only - registered to 10___@u.icesi.edu.co
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (win64 - Windows 11.0 (22621.2))

CPU model: Intel(R) Core(TM) i7-1065G7 CPU @ 1.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads


IIS computed: 4 constraints, 0 bounds
IIS runtime: 0.00 seconds (0.00 work units)
Restricción en IIS: R1_una_vez_LA ORQUIDEA_3_H
Restricción en IIS: R1_una_vez_ALBORADA_3_M
Restricción en IIS: R2_inferior_BLANCO_MEDIANO_2
Restricción en IIS: R2_inferior_BLANCO_MEDIANO_3


In [166]:
#Resultados de la cosecha
if model.status == GRB.OPTIMAL:    
    print("\n📦 Grupos cosechados:")
    for g, p, s in GP:
        for d in D:
            var = x[g, p, s, d]
            if var.X > 0.5:
                print(f"Grupo ({g}, {p}, {s}) cosechado el día {d}")
else: print("\n❌ No hay modelo óptimo")


❌ No hay modelo óptimo


In [167]:
#Desviaciones presentadas y cosecha diaria
if model.status == GRB.OPTIMAL:
    print("\n📉 Desviaciones en el cumplimiento del mix diario:")
    for t in T:
        for c in C:
            for d in D:
                # Calcula la cantidad realmente cosechada para este tipo, categoría y día
                cantidad_cosechada = sum(
                    N[gps] * x[gps + (d,)].X
                    for gps in GP
                    if TIPO[gps] == t and CAT.get(gps + (d,), None) == c
                )
                
                # Obtiene la desviación
                val = dev[t, c, d].X
                
                # Solo muestra si hay desviación significativa (> 0.01)
                if val > 0.01:
                    print(f"Tipo {t}, Categoría {c}, Día {d}: desviación = {val:.0f} aves, cosechado = {cantidad_cosechada:.0f} aves")
else: print("\n❌ No hay modelo óptimo")


❌ No hay modelo óptimo


In [168]:
conteo_mediano = 0
for (g,p,s) in GP:
    for d in D:
        if CAT.get((g,p,s,d)) == "MEDIANO":
            conteo_mediano += 1

print(f"Hay {conteo_mediano} combinaciones grupo-día con categoría MEDIANO")


Hay 350 combinaciones grupo-día con categoría MEDIANO


In [169]:
#Guardar resultados en un CSV
import csv

if model.status == GRB.OPTIMAL:
    with open(f"resultados_cosecha_min{dias:.0f}.csv", "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["Granja", "Galpón", "Sexo", "Día", "Cosechado (1/0)"])
        for g, p, s in GP:
            for d in D:
                writer.writerow([g, p, s, d, int(x[g, p, s, d].X > 0.5)])
else: print("\n❌ No hay modelo óptimo")


❌ No hay modelo óptimo


In [170]:
if model.status == GRB.OPTIMAL:
    with open(f"resultados_cosecha_min{dias:.0f}dias2.csv", "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["Granja", "Galpón", "Sexo", "Día", "Tipo", "Categoría", "Cosechado (1/0)", "Cantidad cosechada"])

        for g, p, s in GP:
            tipo = TIPO[(g, p, s)]
            for d in D:
                cat = CAT.get((g, p, s, d), "NA")
                cosechado = int(x[g, p, s, d].X > 0.5)
                cantidad = int(N[(g, p, s)] * x[g, p, s, d].X + 0.5) if cosechado else 0

                writer.writerow([g, p, s, d, tipo, cat, cosechado, cantidad])
else: print("\n❌ No hay modelo óptimo")


❌ No hay modelo óptimo


### Modelo con la función de maximización de ADG

In [171]:

# === 5. FUNCIÓN OBJETIVO ===

#MAXIMIZAR la ganancia de peso diaria penalizando la desviación de la demanda diaria de pollos
model.setObjective(
    quicksum(
        ADG[g, p, s, d] * N[g, p, s] * x[g, p, s, d]
        for (g, p, s) in GP
        for d in D
        if (g, p, s, d) in ADG
    ) - PENALIZACION * quicksum(dev[t, c, d] for t in T for c in C for d in D),
    GRB.MAXIMIZE
)


In [172]:
# Resolver el modelo
model.optimize()

#Revisar el estado del modelo de optimizacion
if model.status == GRB.OPTIMAL:
    print("\n✅ Solución óptima encontrada.")
elif model.status == GRB.INFEASIBLE:
    print("\n❌ El modelo es infactible.")
    model.computeIIS()
    model.write("modelo_infactible.ilp")
elif model.status == GRB.UNBOUNDED:
    print("\n❌ El modelo no está acotado.")
else:
    print(f"\n⚠️ El modelo terminó con estado: {model.status}")

Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (win64 - Windows 11.0 (22621.2))

CPU model: Intel(R) Core(TM) i7-1065G7 CPU @ 1.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Academic license 2680166 - for non-commercial use only - registered to 10___@u.icesi.edu.co
Optimize a model with 1737 rows, 3148 columns and 19456 nonzeros
Model fingerprint: 0x2e3139c7
Variable types: 208 continuous, 2940 integer (2940 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+04]
  Objective range  [1e+04, 2e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+06]
Presolve removed 1400 rows and 1543 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 0
No other solutions better than -1e+100

Model is infeasible
Best objective -, best bound -, gap -

❌ El modelo es infactible.
Academic l

In [173]:
#Resultados de la cosecha

if model.status == GRB.OPTIMAL:
    print("\n📦 Grupos cosechados:")
    for g, p, s in GP:
        for d in D:
            var = x[g, p, s, d]
            if var.X > 0.5:
                print(f"Grupo ({g}, {p}, {s}) cosechado el día {d}")
else: print("\n❌ No hay modelo óptimo")


❌ No hay modelo óptimo


In [174]:
#Desviaciones presentadas y cosecha diaria

if model.status == GRB.OPTIMAL:
    print("\n📉 Desviaciones en el cumplimiento del mix diario:")
    for t in T:
        for c in C:
            for d in D:
                # Calcula la cantidad realmente cosechada para este tipo, categoría y día
                cantidad_cosechada = sum(
                    N[gps] * x[gps + (d,)].X
                    for gps in GP
                    if TIPO[gps] == t and CAT.get(gps + (d,), None) == c
                )
                
                # Obtiene la desviación
                val = dev[t, c, d].X
                
                # Solo muestra si hay desviación significativa (> 0.01)
                if val > 0.01:
                    print(f"Tipo {t}, Categoría {c}, Día {d}: desviación = {val:.0f} aves, cosechado = {cantidad_cosechada:.0f} aves")
else: print("\n❌ No hay modelo óptimo")


❌ No hay modelo óptimo


In [175]:
if model.status == GRB.INFEASIBLE:    
    model.computeIIS()

    for c in model.getConstrs():
        if c.IISConstr:
            print(f"Restricción en IIS: {c.ConstrName}")


    for v in model.getVars():
        if v.IISLB:
            print(f"Límite inferior en IIS: {v.VarName} ≥ {v.LB}")
        if v.IISUB:
            print(f"Límite superior en IIS: {v.VarName} ≤ {v.UB}")

Academic license 2680166 - for non-commercial use only - registered to 10___@u.icesi.edu.co
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (win64 - Windows 11.0 (22621.2))

CPU model: Intel(R) Core(TM) i7-1065G7 CPU @ 1.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads


IIS computed: 4 constraints, 0 bounds
IIS runtime: 0.00 seconds (0.00 work units)
Restricción en IIS: R1_una_vez_LA ORQUIDEA_3_H
Restricción en IIS: R1_una_vez_ALBORADA_3_M
Restricción en IIS: R2_inferior_BLANCO_MEDIANO_2
Restricción en IIS: R2_inferior_BLANCO_MEDIANO_3


In [176]:
#Guardar resultados en un CSV
import csv
if model.status == GRB.OPTIMAL:
    with open(f"resultados_cosecha_max{dias:.0f}dias.csv", "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["Granja", "Galpón", "Sexo", "Día", "Cosechado (1/0)"])
        for g, p, s in GP:
            for d in D:
                writer.writerow([g, p, s, d, int(x[g, p, s, d].X > 0.5)])
else: print("\n❌ No hay modelo óptimo")


❌ No hay modelo óptimo


In [177]:
import csv

if model.status == GRB.OPTIMAL:
    with open(f"resultados_cosecha_max{dias:.0f}.csv", "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["Granja", "Galpón", "Sexo", "Día", "Tipo", "Categoría", "Cosechado (1/0)", "Cantidad cosechada"])

        for g, p, s in GP:
            tipo = TIPO[(g, p, s)]
            for d in D:
                cat = CAT.get((g, p, s, d), "NA")
                cosechado = int(x[g, p, s, d].X > 0.5)
                cantidad = int(N[(g, p, s)] * x[g, p, s, d].X + 0.5) if cosechado else 0

                writer.writerow([g, p, s, d, tipo, cat, cosechado, cantidad])
else: print("\n❌ No hay modelo óptimo")


❌ No hay modelo óptimo


### Modelo con restricción de cumplimiento semanal por tipo y categoría de peso

In [178]:
# === Calcula semanal de la demanda por categoría

Q_semanal = {}

for (t, c, d), val in Q.items():
    semana = MAPD.get(d)  # Obtener la semana a la que pertenece el día d
    if semana is not None:
        Q_semanal[(t, c, semana)] = Q_semanal.get((t, c, semana), 0) + val


In [179]:
# === R10. RESTRICCION SEMANAL DE DEMANDA DE POLLOS POR TIPO Y CATEGORIA

for t in T:
    for c in ["PEQUEÑO", "MEDIANO"]:  # Solo aplica a estas categorías
        for s in WEEK:
            q_total = Q_semanal.get((t, c, s), 0)

            if q_total > 0:
                dias_semana = [d for d in D if MAPD.get(d) == s]

                suma_cosechada = quicksum(
                    N[gps] * x[gps + (d,)]
                    for gps in GP
                    for d in dias_semana
                    if TIPO[gps] == t and CAT.get(gps + (d,), None) == c
                )

                limite_inferior = (1 - tol) * q_total
                limite_superior = (1 + tol) * q_total

                # Restricción inferior
                model.addConstr(
                    suma_cosechada >= limite_inferior,
                    name=f"R2sem_inferior_{t}_{c}_sem{s}"
                )

                # Restricción superior
                model.addConstr(
                    suma_cosechada <= limite_superior,
                    name=f"R2sem_superior_{t}_{c}_sem{s}"
                )


In [180]:
# Definición de la variable de desviación semanal
desv2 = model.addVars(T, C, WEEK, vtype=GRB.CONTINUOUS, name="desv2")

#Definición de la desviación
for t in T:
    for c in ["PEQUEÑO", "MEDIANO"]:
        for s in WEEK:
            q_total = Q_semanal.get((t, c, s), 0)
            if q_total > 0:
                dias_semana = [d for d in D if MAPD.get(d) == s]

                cantidad_real = quicksum(
                    N[gps] * x[gps + (d,)]
                    for gps in GP
                    for d in dias_semana
                    if TIPO[gps] == t and CAT.get(gps + (d,), None) == c
                )

                model.addConstr(desv2[t, c, s] >= cantidad_real - q_total,
                                name=f"desv_pos_{t}_{c}_sem{s}")
                model.addConstr(desv2[t, c, s] >= q_total - cantidad_real,
                                name=f"desv_neg_{t}_{c}_sem{s}")



In [181]:
# === 5.3. FUNCION OBJETIVO DE MINIMIZACIÓN DE DESVIACIÓN SEMANAL ===

model.setObjective(
    quicksum(desv2[t, c, s] for t in T for c in ["PEQUEÑO", "MEDIANO"] for s in WEEK),
    GRB.MINIMIZE
)


In [182]:
# Resolver el modelo
model.optimize()

#Revisar el estado del modelo de optimizacion
if model.status == GRB.OPTIMAL:
    print("\n✅ Solución óptima encontrada.")
elif model.status == GRB.INFEASIBLE:
    print("\n❌ El modelo es infactible.")
    model.computeIIS()
    model.write("modelo_infactible.ilp")
elif model.status == GRB.UNBOUNDED:
    print("\n❌ El modelo no está acotado.")
else:
    print(f"\n⚠️ El modelo terminó con estado: {model.status}")

Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (win64 - Windows 11.0 (22621.2))

CPU model: Intel(R) Core(TM) i7-1065G7 CPU @ 1.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Academic license 2680166 - for non-commercial use only - registered to 10___@u.icesi.edu.co
Optimize a model with 1753 rows, 3164 columns and 22884 nonzeros
Model fingerprint: 0xd6de22de
Variable types: 224 continuous, 2940 integer (2940 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+06]
Presolve removed 1400 rows and 1555 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.01 work units)
Thread count was 1 (of 8 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -

❌ El modelo es infactible.
Academic license 2680166 - for non-commercial use

In [183]:
#Resultados de la cosecha

if model.status == GRB.OPTIMAL:
    print("\n📦 Grupos cosechados:")
    for g, p, s in GP:
        for d in D:
            var = x[g, p, s, d]
            if var.X > 0.5:
                print(f"Grupo ({g}, {p}, {s}) cosechado el día {d}")
else: print("\n❌ No hay modelo óptimo")


❌ No hay modelo óptimo


In [184]:
#Desviaciones presentadas y cosecha diaria

if model.status == GRB.OPTIMAL:
    print("\n📉 Desviaciones en el cumplimiento del mix diario:")
    for t in T:
        for c in C:
            for d in D:
                # Calcula la cantidad realmente cosechada para este tipo, categoría y día
                cantidad_cosechada = sum(
                    N[gps] * x[gps + (d,)].X
                    for gps in GP
                    if TIPO[gps] == t and CAT.get(gps + (d,), None) == c
                )
                
                # Obtiene la desviación
                val = dev[t, c, d].X
                
                # Solo muestra si hay desviación significativa (> 0.01)
                if val > 0.01:
                    print(f"Tipo {t}, Categoría {c}, Día {d}: desviación = {val:.0f} aves, cosechado = {cantidad_cosechada:.0f} aves")
else: print("\n❌ No hay modelo óptimo")


❌ No hay modelo óptimo


In [185]:
if model.status == GRB.INFEASIBLE:    
    model.computeIIS()

    for c in model.getConstrs():
        if c.IISConstr:
            print(f"Restricción en IIS: {c.ConstrName}")


    for v in model.getVars():
        if v.IISLB:
            print(f"Límite inferior en IIS: {v.VarName} ≥ {v.LB}")
        if v.IISUB:
            print(f"Límite superior en IIS: {v.VarName} ≤ {v.UB}")

Academic license 2680166 - for non-commercial use only - registered to 10___@u.icesi.edu.co
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (win64 - Windows 11.0 (22621.2))

CPU model: Intel(R) Core(TM) i7-1065G7 CPU @ 1.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads


IIS computed: 5 constraints, 0 bounds
IIS runtime: 0.00 seconds (0.00 work units)
Restricción en IIS: R1_una_vez_LA ORQUIDEA_2_MX
Restricción en IIS: R1_una_vez_ALBORADA_3_M
Restricción en IIS: R2_inferior_BLANCO_MEDIANO_1
Restricción en IIS: R2_inferior_BLANCO_MEDIANO_2
Restricción en IIS: R2_inferior_BLANCO_MEDIANO_3


In [186]:
#Guardar resultados en un CSV
import csv
if model.status == GRB.OPTIMAL:
    with open(f"resultados_cosecha_sem_max{dias:.0f}dias.csv", "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["Granja", "Galpón", "Sexo", "Día", "Cosechado (1/0)"])
        for g, p, s in GP:
            for d in D:
                writer.writerow([g, p, s, d, int(x[g, p, s, d].X > 0.5)])
else: print("\n❌ No hay modelo óptimo")


❌ No hay modelo óptimo


In [187]:
import csv

if model.status == GRB.OPTIMAL:
    with open(f"resultados_cosecha_sem_max{dias:.0f}.csv", "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["Granja", "Galpón", "Sexo", "Día", "Tipo", "Categoría", "Cosechado (1/0)", "Cantidad cosechada"])

        for g, p, s in GP:
            tipo = TIPO[(g, p, s)]
            for d in D:
                cat = CAT.get((g, p, s, d), "NA")
                cosechado = int(x[g, p, s, d].X > 0.5)
                cantidad = int(N[(g, p, s)] * x[g, p, s, d].X + 0.5) if cosechado else 0

                writer.writerow([g, p, s, d, tipo, cat, cosechado, cantidad])
else: print("\n❌ No hay modelo óptimo")


❌ No hay modelo óptimo


In [188]:
for c in model.getConstrs():
    print(c.ConstrName)


R1_una_vez_MAO_5_M
R1_una_vez_MAO_6_M
R1_una_vez_LA CHAMBA_1_M
R1_una_vez_LA CHAMBA_2_M
R1_una_vez_LA CHAMBA_3_M
R1_una_vez_LA CHAMBA_4_M
R1_una_vez_CHICO_3_M
R1_una_vez_RANCHO ALEGRE_1_M
R1_una_vez_RANCHO ALEGRE_2_M
R1_una_vez_RANCHO ALEGRE_3_M
R1_una_vez_RANCHO ALEGRE_4_M
R1_una_vez_POTOCO_1_M
R1_una_vez_POTOCO_2_M
R1_una_vez_POTOCO_3_M
R1_una_vez_POTOCO_4_M
R1_una_vez_POTOCO_5_M
R1_una_vez_POTOCO_6_M
R1_una_vez_POTOCO_7_M
R1_una_vez_POTOCO_8_M
R1_una_vez_POTOCO_9_M
R1_una_vez_ARANJUEZ_1_M
R1_una_vez_ARANJUEZ_4_M
R1_una_vez_ARANJUEZ_5_M
R1_una_vez_ARANJUEZ_6_M
R1_una_vez_ARANJUEZ_7_M
R1_una_vez_NIÑA MARIA_1_M
R1_una_vez_NIÑA MARIA_2_M
R1_una_vez_NIÑA MARIA_3_M
R1_una_vez_LA ORQUIDEA_1_M
R1_una_vez_LA ORQUIDEA_2_MX
R1_una_vez_LA ORQUIDEA_3_H
R1_una_vez_MARGARITA_1_M
R1_una_vez_MARGARITA_2_M
R1_una_vez_MARGARITA_3_M
R1_una_vez_MARGARITA_4_M
R1_una_vez_TAMINANGO_1_M
R1_una_vez_TAMINANGO_2_M
R1_una_vez_TAMINANGO_3_M
R1_una_vez_TAMINANGO_4_M
R1_una_vez_SAN BUENO_1_M
R1_una_vez_SAN BUENO_2